In [2]:
#!/usr/bin/env python3
import os
import sys
import glob
import numpy as np
from scipy.io import wavfile
from scipy.signal import resample_poly

# Try GPU (CuPy), otherwise fall back to NumPy
try:
    import cupy as cp
    xp = cp
    rfft = cp.fft.rfft
    asnumpy = cp.asnumpy
    on_gpu = True
    print("Using CuPy on GPU")
except ImportError:
    xp = np
    rfft = np.fft.rfft
    asnumpy = lambda x: x
    on_gpu = False
    print("CuPy not found—falling back to NumPy on CPU")

# Desired parameters
TARGET_SR = 40000       # Hz; ensures Nyquist = 20 000 Hz
DF        = 0.5         # Hz resolution
FFT_LEN   = int(TARGET_SR / DF)  # 40000 / 0.5 = 80 000 samples

# Precompute the full frequency axis (0…20 000 Hz at 0.5 Hz steps)
freqs = np.fft.rfftfreq(FFT_LEN, d=1.0/TARGET_SR)  # length = FFT_LEN/2 + 1

# Find slice indices for 20…20 000 Hz
start_idx = int(20.0 / DF)       # 20 Hz
end_idx   = int(20000.0 / DF)    # 20 000 Hz

def load_and_resample(path):
    sr, data = wavfile.read(path)
    # stereo → mono
    if data.ndim == 2:
        data = data.mean(axis=1)
    data = data.astype(np.float32)
    # resample if needed
    if sr != TARGET_SR:
        data = resample_poly(data, TARGET_SR, sr)
    return data

def process_folder(folder):
    wav_files = glob.glob(os.path.join(folder, '*.wav'))
    if not wav_files:
        print(f"No WAV files found in {folder!r}")
        return

    print(f"Processing {len(wav_files)} files with FFT_LEN={FFT_LEN}, df={DF} Hz")
    for wav_file in wav_files:
        try:
            # 1) Load & ensure correct sampling rate
            data = load_and_resample(wav_file)

            # 2) Pad or truncate to exactly FFT_LEN
            if len(data) < FFT_LEN:
                data = np.pad(data, (0, FFT_LEN - len(data)), 'constant')
            else:
                data = data[:FFT_LEN]

            # 3) Compute real-FFT and magnitude
            arr      = xp.asarray(data)
            fft_vals = rfft(arr, n=FFT_LEN)   # length = FFT_LEN/2 + 1
            mag      = xp.abs(fft_vals)

            # 4) Bring back to CPU if needed
            mag = asnumpy(mag)

            # 5) Slice to 20…20 000 Hz
            sel_freqs = freqs[start_idx:end_idx+1]
            sel_mag   = mag[start_idx:end_idx+1]

            # 6) Save to CSV
            out_csv = os.path.splitext(wav_file)[0] + '.csv'
            header  = 'frequency_hz,magnitude'
            np.savetxt(
                out_csv,
                np.column_stack((sel_freqs, sel_mag)),
                delimiter=',',
                header=header,
                comments='',
                fmt='%.2f'
            )
            print(f"✔ Saved {out_csv} ({len(sel_freqs)} bins)")
        except Exception as e:
            print(f"✖ Error processing {wav_file}: {e}")

if __name__ == '__main__':
    folder = './for-norm/for-norm/testing/fake/'
    process_folder(folder)

Using CuPy on GPU
Processing 2370 files with FFT_LEN=80000, df=0.5 Hz
✔ Saved ./for-norm/for-norm/testing/fake\file1.wav_16k.wav_norm.wav_mono.wav_silence.csv (39961 bins)
✔ Saved ./for-norm/for-norm/testing/fake\file10.wav_16k.wav_norm.wav_mono.wav_silence.csv (39961 bins)
✔ Saved ./for-norm/for-norm/testing/fake\file100.wav_16k.wav_norm.wav_mono.wav_silence.csv (39961 bins)
✔ Saved ./for-norm/for-norm/testing/fake\file1000.wav_16k.wav_norm.wav_mono.wav_silence.csv (39961 bins)
✔ Saved ./for-norm/for-norm/testing/fake\file1001.wav_16k.wav_norm.wav_mono.wav_silence.csv (39961 bins)
✔ Saved ./for-norm/for-norm/testing/fake\file1002.wav_16k.wav_norm.wav_mono.wav_silence.csv (39961 bins)
✔ Saved ./for-norm/for-norm/testing/fake\file1003.wav_16k.wav_norm.wav_mono.wav_silence.csv (39961 bins)
✔ Saved ./for-norm/for-norm/testing/fake\file1004.wav_16k.wav_norm.wav_mono.wav_silence.csv (39961 bins)
✔ Saved ./for-norm/for-norm/testing/fake\file1005.wav_16k.wav_norm.wav_mono.wav_silence.csv (399